### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [3]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [4]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [5]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [6]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4090 - loss: 0.8780    
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6996 - loss: 0.5700
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3808 - loss: 0.9120
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6786 - loss: 0.5777
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6314 - loss: 0.6371
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6511 - loss: 0.6131
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7387 - loss: 0.5595
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6620 - loss: 0.6331
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5693 - loss: 0.726115
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - accuracy: 0.7776 - loss: 0.5317
Epoch 3/50
167/167 ━━━━━━━━━━━━━━

/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._

Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - accuracy: 0.8677 - loss: 0.3156
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 65/167 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.8786 - loss: 0.3107Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  ep - accuracy: 0.8741 - loss: 0.314
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.8729 - loss: 0.3152
Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Epoch 1/50
 1/84 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/stepEpoch 1/50


...

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8955 - loss: 0.2536
Epoch 48/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8790 - loss: 0.2981  
Epoch 100/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8956 - loss: 0.2559  
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8834 - loss: 0.2726
Epoch 50/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - accuracy: 0.8751 - loss: 0.3006
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8644 - loss: 0.3268
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8813 - loss: 0.2835
Epoch 55/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8606 - loss: 0.3435
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8760 - loss: 0.3012
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9013 - loss: 0.2256
Epoch 50/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8930 - loss: 0.25929
Epoch 49/100
84/84 ━━━━━━━━━━━━━━━━━━

/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.8841 - loss: 0.2856 
Epoch 57/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8642 - loss: 0.3258  
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8937 - loss: 0.2426  
Epoch 52/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8939 - loss: 0.2515
Epoch 51/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8616 - loss: 0.3286
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - accuracy: 0.8810 - loss: 0.2823
Epoch 52/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7861 - loss: 0.4832
Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8778 - loss: 0.2865  
Epoch 58/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.8654 - loss: 0.3261
Epoch 9/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - accuracy: 0.9033 - loss: 0.2195
Epoch 53/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8865 - loss: 0.2628
Epoch 52/100
167/167 ━━━━━━━━━━━

/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.7576 - loss: 0.5449
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - accuracy: 0.7929 - loss: 0.4630
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step - accuracy: 0.8111 - loss: 0.4308
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.8186 - loss: 0.4216
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - accuracy: 0.8202 - loss: 0.4181
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.8307 - loss: 0.3982
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.8419 - loss: 0.3856
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.8417 - loss: 0.3864
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.8418 - loss: 0.3800
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.8448 - loss: 0.3720
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.8520 - loss: 0.3591
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 